In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image
 
 
model = load_model('model.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 36,380,161
Non-trainable params: 53,376
_________________________________________________________________


In [2]:
import face_recognition
import cv2
import numpy as np

In [3]:
cd maskdata/knowns

C:\Users\kimhy\K-Digital MLDL\Semi-project3\4팀\maskdata\knowns


In [4]:
known_face_dir = os.listdir('./')

In [5]:
# 아는 얼굴들 미리 모아놓기

known_face_encodings = []
known_user_names = []
error_user_list = []
for known_face_user in known_face_dir:
    known_user_image = face_recognition.load_image_file(known_face_user)
    try:
        known_user_encoding = face_recognition.face_encodings(known_user_image)[0]
        known_face_encodings.append(known_user_encoding)
        known_user_names.append(known_face_user[:-4])
    except: # 얼굴인식이 안되는 사진들은 리스트에 넣어지지않도록 따로 분리
        print(f"{known_face_user[:-4]}의 얼굴인식이 되지 않습니다.재촬영이 필요합니다.")
        error_user_list.append(known_face_user)

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image

In [7]:
# 아는 얼굴 찾기
def known_face_name(frame, known_face_encodings, known_user_names):
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    name = ''
    # 얼굴 인식 처리 속도를 높이기 위해 비디오 프레임 크기를 1/4 크기로 조정
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # 영상을 BGR 색상(OpenCV가 사용하는)에서 RGB 색상(페이스_인식 사용)으로 변환
    rgb_small_frame = small_frame[:, :, ::-1]

    # 다른 모든 비디오 프레임만 처리하여 시간 절약
    if process_this_frame:
        # 현재 비디오 프레임에서 모든 얼굴 및 얼굴 인코딩 찾기
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # 얼굴이 known face(s)와 일치하는지 확인
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # known_face_encodings에서 일치하는 항목이 발견되면 첫 번째 항목을 사용
            # 일치하는 경우:
            # first_match_index =이(가) 일치합니다.index(참)
            # name = knowled_face_names[first_match_index]

            # 대신 새 얼굴까지의 거리가 가장 작은 known 얼굴을 사용
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_user_names[best_match_index]

    
        return name

In [9]:
cd ../

C:\Users\kimhy\K-Digital MLDL\Semi-project3\4팀\maskdata


In [10]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 80B0-AEF4

 C:\Users\kimhy\K-Digital MLDL\Semi-project3\4팀\maskdata 디렉터리

2021-01-31  오후 01:40    <DIR>          .
2021-01-31  오후 01:40    <DIR>          ..
2021-01-31  오후 01:41    <DIR>          face_recognition
2021-01-31  오후 12:44    <DIR>          knowns
2021-01-31  오후 12:36    <DIR>          Mask
2021-01-31  오후 02:59    <DIR>          nomask_capture
2021-01-31  오후 12:36    <DIR>          Non_Mask
2021-01-31  오후 12:36    <DIR>          unknown_face_classifier
               0개 파일                   0 바이트
               8개 디렉터리  23,803,576,320 바이트 남음


In [15]:
from datetime import datetime
# open webcam
webcam = cv2.VideoCapture(0)
 
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
 
# loop through frames
while webcam.isOpened():
 
    # read frame from webcam 
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
 
            if prediction < 0.5: # 마스크 미착용으로 판별되면,
                result_name = known_face_name(frame, known_face_encodings, known_user_names)
#                 print(result_name)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                result_text = result_name + " is " + text
                cv2.putText(frame, result_text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                nomask_capture_path = './nomask_capture/'
                now_time = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')[:-1] # 10초마다 한번씩
                name_path = nomask_capture_path + result_name + '/'
                if result_name == "Unknown":
                    if not os.path.isdir(name_path): 
                        os.mkdir(result_name)
                else:
                    if not os.path.isdir(name_path):
                        os.mkdir(result_name)
                        
                check_img_name = f"{result_name}{now_time}.png"
                img_name = f"{name_path}{result_name}{now_time}.png"
#                 if is not os.path.isfile(check_img_name):
#                     cv2.imwrite(img_name, frame)
#                 else:
#                     pass
                print(img_name)
                cv2.imwrite(img_name, frame)
                os.chdir('../../')
                
            else: # 마스크 착용으로 판별되면
                result_name = known_face_name(frame, known_face_encodings, known_user_names)
#                 print(result_name)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                result_text = result_name + " is " + text
                cv2.putText(frame, result_text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                
    # display output
    cv2.imshow("mask nomask classify", frame)
 
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows() 


./nomask_capture/hyojin/hyojin2021_01_31.png
./nomask_capture/Unknown/Unknown2021_01_31.png
./nomask_capture/hyojin/hyojin2021_01_31.png


FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'hyojin'

In [13]:
# 에러발생시
# webcam.release()
# cv2.destroyAllWindows() 